In [1]:
import os, nltk, codecs
from nltk import word_tokenize
from time import time
import sys
t0 = time()
def my_dummy(doc):
    return doc

In [2]:
#Load Data to pandas - frame vor better visualization
import pandas as pd
complete_data = pd.read_csv('omt_shortned.txt', sep = ";",
                           names = ["#","Answers", "Class" , "Subclass"] )
print('before cleaning...\n')
print(complete_data.info(), "\n")
print(complete_data["Class"].value_counts(), "\n")
#delete eventually empty entries
complete_data = complete_data.dropna(axis = 0)
#delete eventually wrong labels ('\N' seems to accure from time to time)
complete_data = complete_data.drop(complete_data[complete_data.Class == '\\N'].index).reset_index()
print('after cleaning...\n')
print(complete_data.info())
print(complete_data["Class"].value_counts(), "\n")
print(complete_data.head)

before cleaning...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
#           1000 non-null object
Answers     1000 non-null object
Class       1000 non-null object
Subclass    1000 non-null object
dtypes: object(4)
memory usage: 31.3+ KB
None 

M     518
L     194
A     192
0      91
\N      5
Name: Class, dtype: int64 

after cleaning...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 995 entries, 0 to 994
Data columns (total 5 columns):
index       995 non-null int64
#           995 non-null object
Answers     995 non-null object
Class       995 non-null object
Subclass    995 non-null object
dtypes: int64(1), object(4)
memory usage: 38.9+ KB
None
M    518
L    194
A    192
0     91
Name: Class, dtype: int64 

<bound method NDFrame.head of      index                               #  \
0        0        185124138106620081063215   
1        1        185124138106620081063215   
2        2        185124138106620081063215   
3

In [24]:
from sklearn.model_selection import train_test_split

# extract the needed infos from data_complete
data = complete_data.filter(["Answers", "Class"],axis = 1)

#split it into X ( = input) and y ( the labels)
X, y = data["Answers"], data["Class"]

#split data into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size = 0.3, random_state = 42)


#Bei größerem Trainingsset: stratified_shufflesplit nutzen

In [26]:
#preprocessing
import re
from nltk.corpus import stopwords
import numpy as np

# The pattern I don't want in my corpus 
pattern = re.compile("\-|\!|\:|\.|\d|\,|\:|\(|\)|\?|\"|\\|\+|\%|\/")

#return: a patternless copy of the input string
def insert_space_between_special_chars(strg):
    return re.sub('([a-zA-Z])([^A-Za-z0-9]+)([a-zA-Z])', r'\1\2 \3', strg)

def remove_patterns_from_string(strg):
    return re.sub(pattern, " ", strg)

# german stopwords (words who accure very often in texts and don't contribute anything to the text like articles, conjunctions)
stopWords = set(stopwords.words("german"))
stopWords.add('dass')

#tokenizes a list of strings
#return: list of tokens
def tokenize_text(text):
    tokens = nltk.word_tokenize(text, language = 'german')
    return tokens

#retrns: list of strings which do not occure in stopWords
def remove_stopwords(text):
    return [w for w in text if not w in stopWords]

# in : row for row from vector
# out: preprocessed version of the row
def preprocess_text(corpus):
    normalized_corpus = []
    for text in corpus:
        #to change xyz.abc to xyz. abc
        text = re.sub('([a-zA-Z])(\.)([a-zA-Z])', r'\1\2 \3', text)
        text = text.lower()
        #text = remove_patterns_from_string(text)
        text = tokenize_text(text)
        text = remove_stopwords(text)
        normalized_corpus.append(text)
    return normalized_corpus

#later preprocess only X_train and X_val. X_test gets own pipeline in the end
X_train_preprocessed = preprocess_text(X_train)
X_test_preprocessed = preprocess_text(X_test)


print(X_train_preprocessed[:2])
print(type(X_train_preprocessed))
print(type(y_train))

[['person', 'kurz', 'feierabend', 'unterhält', 'tag', '.', 'weiterzuarbeiten', 'gespräch', 'gleichwichtig', '.', 'unterhalten', 'erschöpft', '.', 'geschichte', 'person', 'anhört', 'gearbeitet', 'hat/noch', '.', 'gespräch', 'kollegen', 'bald', 'ende', 'person', 'macht', 'restliche', 'arbeit', 'fertig', 'geht', 'hause', '.'], ['macht', 'grösser', 'wichtig', '.', 'dominant', 'recht', '.', 'körperhaltung', ':', 'steht', ',', 'angewinkelte', 'arme', ',', 'kopf', 'leicht', 'zweiten', 'person', 'geneigt', '.', 'setzt', 'löst', 'augenhöhe', '.']]
<class 'list'>
<class 'pandas.core.series.Series'>


In [5]:
from sklearn.metrics import classification_report
from yellowbrick.classifier import ClassificationReport
import pandas as pd

# minimal classification performance: all answers M
m_labels = []
for i in range(len(y_test)):
    m_labels.append("M")
machtmotiv = pd.Series(m_labels)
#plot_classification_report(classification_report(y_test, machtmotiv))
print(classification_report(y_test, machtmotiv))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        40
           A       0.00      0.00      0.00        65
           L       0.00      0.00      0.00        71
           M       0.56      1.00      0.72       222

   micro avg       0.56      0.56      0.56       398
   macro avg       0.14      0.25      0.18       398
weighted avg       0.31      0.56      0.40       398



C:\Users\fmeyer\Anaconda3\envs\classification_omt\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [6]:
from sklearn.datasets import fetch_20newsgroups
# Load some categories from the training set
categories = [
    'alt.atheism',
    'talk.religion.misc',
]
# Uncomment the following to do the analysis on all the categories
#categories = None

print("Loading 20 newsgroups dataset for categories:")
print(categories)

data = fetch_20newsgroups(subset='train', categories=categories)
print("%d documents" % len(data.filenames))
print("%d categories" % len(data.target_names))
print()
print(type(data))

Loading 20 newsgroups dataset for categories:
['alt.atheism', 'talk.religion.misc']
857 documents
2 categories

<class 'sklearn.utils.Bunch'>


In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn. decomposition import LatentDirichletAllocation
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline, FeatureUnion


print("start classification with SVC...")
#define pipeline combining feature extractor with classifier
pipeline1 = Pipeline([
    ('vect1', CountVectorizer(preprocessor = my_dummy,
                              tokenizer =my_dummy, 
                              token_pattern=None)),
                            
    ('tf1',TfidfTransformer()),
                            
    ('clf1', SVC()),    
])

parameters1 = {
    'vect1__max_df': (0.9, 1.0),
    'vect1__ngram_range':((1,1),(1,2),(1,3)),
    'vect1__min_df': (1,2),
    'tf1__use_idf': (True, False),
    'clf1__kernel': ('linear','rbf'), 
    'clf1__C':(0.001,0.1,1,10,100),
    'clf1__max_iter':(50,100,200),
}

if __name__ == "__main__":
    #find best params for this feature extraction methods
    grid_search1 = GridSearchCV(pipeline1, parameters1, cv=5,
                           n_jobs = -1, verbose = 1)
    print("performing gridsearch...")
    print("pipeline:", [name for name, _ in pipeline1.steps])
    print("parameters:")
    print(parameters1)
    time_svc = time()
    grid_search1.fit(X_train_preprocessed, y_train)
    print("done in %0.3fs" % (time()-time_svc))
    print()

    print("Best score: %0.3f" % grid_search1.best_score_)
    print("best parameters set:")
    best_parameters = grid_search1.best_estimator_.get_params()
    for param_name in sorted(parameters1.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))


start classification with SVC...
performing gridsearch...
pipeline: ['vect1', 'tf1', 'clf1']
parameters:
{'vect1__max_df': (0.9, 1.0), 'vect1__ngram_range': ((1, 1), (1, 2), (1, 3)), 'vect1__min_df': (1, 2), 'tf1__use_idf': (True, False), 'clf1__kernel': ('linear', 'rbf'), 'clf1__C': (0.001, 0.1, 1, 10, 100), 'clf1__max_iter': (50, 100, 200)}
Fitting 5 folds for each of 720 candidates, totalling 3600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   10.8s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   23.1s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   41.5s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:  2.9min


done in 199.881s

Best score: 0.714
best parameters set:
	clf1__C: 10
	clf1__kernel: 'linear'
	clf1__max_iter: 100
	tf1__use_idf: False
	vect1__max_df: 0.9
	vect1__min_df: 1
	vect1__ngram_range: (1, 3)


[Parallel(n_jobs=-1)]: Done 3600 out of 3600 | elapsed:  3.3min finished
C:\Users\fmeyer\Anaconda3\envs\classification_omt\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\fmeyer\Anaconda3\envs\classification_omt\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn. decomposition import LatentDirichletAllocation
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, FeatureUnion


print("start classification with SVC...")
#define pipeline combining feature extractor with classifier
pipeline2 = Pipeline([
    ('vect2', CountVectorizer(preprocessor = my_dummy,
                              tokenizer =my_dummy, 
                              token_pattern=None)),
                            
    ('tf2',TfidfTransformer()),
                            
    ('clf2', KNeighborsClassifier()),    
])

parameters2 = {
    'vect2__max_df': (0.9,0.95,1.0),
    'vect2__ngram_range':((1,1),(1,2),(1,3)),
    'vect2__min_df': (1,2,5),
    'tf2__use_idf': (True, False),
    'clf2__n_neighbors': (5,10), 
    'clf2__algorithm':('ball_tree', 'kd_tree'),
    'clf2__leaf_size':(30,50,80),
}

if __name__ == "__main__":
    #find best params for this feature extraction methods
    grid_search2 = GridSearchCV(pipeline2, parameters2, cv=5,
                           n_jobs = -1, verbose = 1)
    print("performing gridsearch...")
    print("pipeline:", [name for name, _ in pipeline2.steps])
    print("parameters:")
    print(parameters2)
    time_svc = time()
    grid_search2.fit(X_train_preprocessed, y_train)
    print("done in %0.3fs" % (time()-time_svc))
    print()

    print("Best score: %0.3f" % grid_search2.best_score_)
    print("best parameters set:")
    best_parameters = grid_search2.best_estimator_.get_params()
    for param_name in sorted(parameters2.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))


start classification with SVC...
performing gridsearch...
pipeline: ['vect2', 'tf2', 'clf2']
parameters:
{'vect2__max_df': (0.9, 0.95, 1.0), 'vect2__ngram_range': ((1, 1), (1, 2), (1, 3)), 'vect2__min_df': (1, 2, 5), 'tf2__use_idf': (True, False), 'clf2__n_neighbors': (5, 10), 'clf2__algorithm': ('ball_tree', 'kd_tree'), 'clf2__leaf_size': (30, 50, 80)}
Fitting 5 folds for each of 648 candidates, totalling 3240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   19.7s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   33.9s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:   52.6s
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:  2.3min


done in 138.179s

Best score: 0.688
best parameters set:
	clf2__algorithm: 'ball_tree'
	clf2__leaf_size: 30
	clf2__n_neighbors: 10
	tf2__use_idf: True
	vect2__max_df: 0.9
	vect2__min_df: 2
	vect2__ngram_range: (1, 2)


[Parallel(n_jobs=-1)]: Done 3240 out of 3240 | elapsed:  2.3min finished
C:\Users\fmeyer\Anaconda3\envs\classification_omt\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\fmeyer\Anaconda3\envs\classification_omt\lib\site-packages\sklearn\neighbors\base.py:217: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "


In [29]:
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn. decomposition import LatentDirichletAllocation
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.naive_bayes import MultinomialNB


print("start classification with MNB and extra LDA - Features...")
#define pipeline combining feature extractor with classifier
pipeline3 = Pipeline([
    ('union', FeatureUnion(n_jobs = -1,
                          transformer_list = [
                              ('pipe1', Pipeline([
                                  ('vect3', CountVectorizer(preprocessor = my_dummy,
                                                              tokenizer =my_dummy, 
                                                              token_pattern=None)),
                                  ('tf3',TfidfTransformer()),
                            
    
                              ])),
                              ('pipe2', Pipeline([
                                  ('vect_lda', CountVectorizer(preprocessor = my_dummy,
                                                               tokenizer =my_dummy, token_pattern=None)),
                                   ('lda1', LatentDirichletAllocation(max_iter = 100, learning_method = 'online', 
                                                                      learning_offset=50))
                              ]))
                            ])),
                            
    ('clf3', MultinomialNB()),    
])

parameters3 = {
    'union__pipe1__vect3__max_df': (0.9,0.95,1.0),
    'union__pipe1__vect3__ngram_range':((1,1),(1,2),(1,3)),
    'union__pipe1__vect3__min_df': (1,2,5),
    'union__pipe1__tf3__use_idf': (True, False),
    'union__pipe2__vect_lda__ngram_range':((1,1),(1,2)),
    'union__pipe2__lda1__n_components': (3,6,10),
    }

if __name__ == "__main__":
    #find best params for this feature extraction methods
    grid_search3 = GridSearchCV(pipeline3, parameters3, cv=5,
                           n_jobs = -1, verbose = 1)
    print("performing gridsearch...")
    print("pipeline:", [name for name, _ in pipeline3.steps])
    print("parameters:")
    print(parameters3)
    time_svc = time()
    grid_search3.fit(X_train_preprocessed, y_train)
    print("done in %0.3fs" % (time()-time_svc))
    print()

    print("Best score: %0.3f" % grid_search3.best_score_)
    print("best parameters set:")
    best_parameters = grid_search3.best_estimator_.get_params()
    for param_name in sorted(parameters3.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))


start classification with MNB and extra LDA - Features...
performing gridsearch...
pipeline: ['union', 'clf3']
parameters:
{'union__pipe1__vect3__max_df': (0.9, 0.95, 1.0), 'union__pipe1__vect3__ngram_range': ((1, 1), (1, 2), (1, 3)), 'union__pipe1__vect3__min_df': (1, 2, 5), 'union__pipe1__tf3__use_idf': (True, False), 'union__pipe2__vect_lda__ngram_range': ((1, 1), (1, 2)), 'union__pipe2__lda1__n_components': (3, 6, 10)}
Fitting 5 folds for each of 324 candidates, totalling 1620 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   54.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 11.6min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed: 20.9min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed: 33.8min
[Parallel(n_jobs=-1)]: Done 1620 out of 1620 | elapsed: 46.1min finished
C:\Users\fmeyer\Anaconda3\envs\classification_omt\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


done in 2774.454s

Best score: 0.652
best parameters set:
	union__pipe1__tf3__use_idf: True
	union__pipe1__vect3__max_df: 0.9
	union__pipe1__vect3__min_df: 5
	union__pipe1__vect3__ngram_range: (1, 1)
	union__pipe2__lda1__n_components: 6
	union__pipe2__vect_lda__ngram_range: (1, 2)


In [33]:
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn. decomposition import LatentDirichletAllocation
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC


print("start classification with MNB and extra LDA - Features...")
#define pipeline combining feature extractor with classifier
pipeline4 = Pipeline([
    ('union1', FeatureUnion(n_jobs = -1,
                          transformer_list = [
                              ('pipe3', Pipeline([
                                  ('vect4', CountVectorizer(preprocessor = my_dummy,
                                                              tokenizer =my_dummy, 
                                                              token_pattern=None)),
                                  ('tf4',TfidfTransformer()),
                            
    
                              ])),
                              ('pipe4', Pipeline([
                                  ('vect_lda1', CountVectorizer(preprocessor = my_dummy,
                                                               tokenizer =my_dummy, token_pattern=None)),
                                   ('lda2', LatentDirichletAllocation(max_iter = 100, learning_method = 'online'))
                              ]))
                            ])),
                            
    ('clf4', SVC()),    
])

parameters4 = {
    'union1__pipe3__vect4__max_df': (0.9,0.95,1.0),
    'union1__pipe3__vect4__ngram_range':((1,1),(1,2),(1,3)),
    'union1__pipe3__vect4__min_df': (1,2,5),
    'union1__pipe3__tf4__use_idf': (True, False),
    'union1__pipe4__vect_lda1__ngram_range':((1,1),),
    'union1__pipe4__lda2__n_components': (3,6),
    'union1__pipe4__lda2__learning_offset': (5,10,20),
    'clf4__kernel': ('linear',), 
    'clf4__C':(0.1,1,10,100),
    'clf4__max_iter':(100,200),
    }

if __name__ == "__main__":
    #find best params for this feature extraction methods
    grid_search4 = GridSearchCV(pipeline4, parameters4, cv=5,
                           n_jobs = -1, verbose = 1)
    print("performing gridsearch...")
    print("pipeline:", [name for name, _ in pipeline4.steps])
    print("parameters:")
    print(parameters4)
    time_svc = time()
    grid_search4.fit(X_train_preprocessed, y_train)
    print("done in %0.3fs" % (time()-time_svc))
    print()

    print("Best score: %0.3f" % grid_search4.best_score_)
    print("best parameters set:")
    best_parameters = grid_search4.best_estimator_.get_params()
    for param_name in sorted(parameters4.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))


start classification with MNB and extra LDA - Features...
performing gridsearch...
pipeline: ['union1', 'clf4']
parameters:
{'union1__pipe3__vect4__max_df': (0.9, 0.95, 1.0), 'union1__pipe3__vect4__ngram_range': ((1, 1), (1, 2), (1, 3)), 'union1__pipe3__vect4__min_df': (1, 2, 5), 'union1__pipe3__tf4__use_idf': (True, False), 'union1__pipe4__vect_lda1__ngram_range': ((1, 1),), 'union1__pipe4__lda2__n_components': (3, 6), 'union1__pipe4__lda2__learning_offset': (5, 10, 20), 'clf4__kernel': ('linear',), 'clf4__C': (0.1, 1, 10, 100), 'clf4__max_iter': (100, 200)}
Fitting 5 folds for each of 2592 candidates, totalling 12960 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 12.8min


KeyboardInterrupt: 